# Traitement des données de colonnes

Dans le cadre de ce module, nous explorerons les fonctions disponibles sous `org.apache.spark.sql.functions` pour dériver de nouvelles valeurs à partir de valeurs de colonnes existantes dans un Data Frame.

## Introduction aux fonctions prédéfinies

Les données dans les colonnes sont généralement traitées à l'aide de fonctions issues de `org.apache.spark.sql.functions`. Nous allons comprendre les détails de ces fonctions en détail.
* Récapitulons les Fonctions ou les API pour traiter les Data Frames.
 * Projection - `select` ou `withColumn`
 * Filtrage - `filter` ou `where`
 * Regrouper les données par clé et effectuer des agrégations - `groupBy`
 * Tri des données - `sort` ou `orderBy` 
* Nous pouvons transmettre des noms de colonnes, des littéraux ou des expressions à toutes les API Data Frame.
* Les expressions comprennent des opérations arithmétiques, des transformations utilisant des fonctions de `org.apache.spark.sql.functions`.
* Il y a environ 300 fonctions sous `org.apache.spark.sql.functions`.
* Nous parlerons de certaines des fonctions importantes utilisées pour la manipulation de chaînes, la manipulation de dates, etc.

In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.
    builder.
    appName("Processing Column Data").
    master("local[*]").
    getOrCreate

spark = org.apache.spark.sql.SparkSession@324087d6


org.apache.spark.sql.SparkSession@324087d6

Voici un exemple de création de Data Frame à l'aide d'une collection d'employés. Nous utiliserons ce Data Frame pour explorer toutes les fonctions importantes pour traiter en détail les données des colonnes.

In [3]:
val employees = List((1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                    )

employees = List((1,Scott,Tiger,1000.0,united states,+1 123 456 7890,123 45 6789), (2,Henry,Ford,1250.0,India,+91 234 567 8901,456 78 9123), (3,Nick,Junior,750.0,united KINGDOM,+44 111 111 1111,222 33 4444), (4,Bill,Gomes,1500.0,AUSTRALIA,+61 987 654 3210,789 12 6118))


List((1,Scott,Tiger,1000.0,united states,+1 123 456 7890,123 45 6789), (2,Henry,Ford,1250.0,India,+91 234 567 8901,456 78 9123), (3,Nick,Junior,750.0,united KINGDOM,+44 111 111 1111,222 33 4444), (4,Bill,Gomes,1500.0,AUSTRALIA,+61 987 654 3210,789 12 6118))

In [4]:
employees.size

4

In [5]:
val employeesDF = employees.
    toDF("employee_id", "first_name",
         "last_name", "salary",
         "nationality", "phone_number",
         "ssn"
        )

employeesDF = [employee_id: int, first_name: string ... 5 more fields]


[employee_id: int, first_name: string ... 5 more fields]

In [6]:
employeesDF.printSchema

root
 |-- employee_id: integer (nullable = false)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- salary: double (nullable = false)
 |-- nationality: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- ssn: string (nullable = true)



In [8]:
employeesDF.show()

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+--------------+----------------+-----------+



## Catégories de fonctions

Il existe environ 300 fonctions sous org.apache.spark.sql.functions. À un niveau supérieur, ils peuvent être regroupés en quelques catégories.
* Fonctions de manipulation de chaînes
 * Conversion de la casse - `lower`,  `upper`
 * Obtenir la longueur -  `length`
 * Extraction de sous-chaînes - `substring`, `split`
 * Trimming - `trim`, `ltrim`, `rtrim`
 * Padding - `lpad`, `rpad`
 * Concaténer des chaînes - `concat`
* Fonctions de manipulation de date
 * Obtenir la date et l'heure actuelles - `current_date`, `current_timestamp`
 * Arithmétique des dates - `date_add`, `date_sub`, `datediff`, `months_between`, `add_months`, `next_day`
 * Date ou heure de début et de fin - `last_day`, `trunc`, `date_trunc`
 * Mise en forme de date - `date_format`
 * Extraction d'informations - `dayofyear`, `dayofmonth`, `dayofweek`, `year`, `month`
* Fonctions d'agrégation
 * `count`, `countDistinct`
 * `sum`, `avg`
 * `min`, `max`
* Autres fonctions - Nous explorerons en fonction des cas d'utilisation.

## Fonctions spéciales - col and lit

Comprenons les fonctions spéciales telles que col et lit.

* Pour les API Data Frame telles que `select`, `groupBy`, `orderBy` etc nous pouvons passer des noms de colonnes sous forme de chaînes.

In [9]:
// Alternative by passing column names as strings.
employeesDF.
    select("first_name", "last_name").
    show

+----------+---------+
|first_name|last_name|
+----------+---------+
|     Scott|    Tiger|
|     Henry|     Ford|
|      Nick|   Junior|
|      Bill|    Gomes|
+----------+---------+



In [10]:
// Alternative using col function
// $ is shorthand operator for col from implicits
import org.apache.spark.sql.functions.col

In [11]:
employeesDF.
    select(col("first_name"), col("last_name")).
    show

+----------+---------+
|first_name|last_name|
+----------+---------+
|     Scott|    Tiger|
|     Henry|     Ford|
|      Nick|   Junior|
|      Bill|    Gomes|
+----------+---------+



In [12]:
// to use operators such as $ in place of functions like col
import spark.implicits._

In [13]:
employeesDF.
    select($"first_name", $"last_name").
    show

+----------+---------+
|first_name|last_name|
+----------+---------+
|     Scott|    Tiger|
|     Henry|     Ford|
|      Nick|   Junior|
|      Bill|    Gomes|
+----------+---------+



* S'il n'y a aucune transformation sur une colonne dans une fonction, nous pouvons transmettre tous les noms de colonne sous forme de chaînes.
* Sinon, nous devons transmettre toutes les colonnes en tant que type de colonne en utilisant la fonction col ou son opérateur abrégé $.


In [15]:
// Passing columns as part of groupBy
employeesDF.
    groupBy("nationality").
    count.
    show

+--------------+-----+
|   nationality|count|
+--------------+-----+
| united states|    1|
|         India|    1|
|united KINGDOM|    1|
|     AUSTRALIA|    1|
+--------------+-----+



In [16]:
import org.apache.spark.sql.functions.upper

In [23]:
employeesDF.
    groupBy(upper(col("nationality"))).
    count.
    show

+------------------+-----+
|upper(nationality)|count|
+------------------+-----+
|     UNITED STATES|    1|
|             INDIA|    1|
|    UNITED KINGDOM|    1|
|         AUSTRALIA|    1|
+------------------+-----+



In [26]:
// Passing columns as part of orderBy or sort
employeesDF.
    orderBy("employee_id").
    show

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+--------------+----------------+-----------+



Cependant, si nous voulons appliquer une transformation à l'aide de fonctions, il ne suffira pas de transmettre des noms de colonne sous forme de chaînes à certaines fonctions. Nous devons les passer comme type de colonne.

In [27]:
//This code fails as upper is not valid function on string
employeesDF.
    select(upper("first_name")).
    show

Unknown Error: <console>:34: error: type mismatch;
 found   : String("first_name")
 required: org.apache.spark.sql.Column
           select(upper("first_name")).
                        ^


* `col` est la fonction qui convertira le nom de la colonne du type chaîne en type **Colonne**. Nous pouvons également faire référence aux noms de colonne en tant que type ** Colonne ** en utilisant le nom du Data Frame.

In [28]:
// Alternate using $ and upper
employeesDF.
    select(upper($"first_name")).
    show

+-----------------+
|upper(first_name)|
+-----------------+
|            SCOTT|
|            HENRY|
|             NICK|
|             BILL|
+-----------------+



In [29]:
// Using as part of groupBy
employeesDF.
    groupBy(upper($"nationality")).
    count.
    show

+------------------+-----+
|upper(nationality)|count|
+------------------+-----+
|     UNITED STATES|    1|
|             INDIA|    1|
|    UNITED KINGDOM|    1|
|         AUSTRALIA|    1|
+------------------+-----+



In [30]:
// Using as part of orderBy
employeesDF.
    orderBy(upper($"nationality")).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
+-----------+----------+---------+------+--------------+----------------+-----------+



In [31]:
// Alternative - we can also refer column names using Data Frame like this
employeesDF.
    orderBy(upper(employeesDF("nationality"))).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
+-----------+----------+---------+------+--------------+----------------+-----------+



* Parfois, nous souhaitons ajouter un littéral aux valeurs des colonnes. Par exemple, nous pourrions vouloir concaténer first_name et last_name séparés par une virgule et un espace entre eux.

In [32]:
import org.apache.spark.sql.functions.concat

In [ ]:
// Les approches ci-dessous échouent.

In [33]:
employeesDF.
    select(concat($"first_name", ", ", $"last_name")).
    show()

Unknown Error: <console>:34: error: type mismatch;
 found   : String(", ")
 required: org.apache.spark.sql.Column
           select(concat($"first_name", ", ", $"last_name")).
                                        ^


In [34]:
// Same as above
employeesDF.
    select(concat(col("first_name"), ", ", col("last_name"))).
    show

Unknown Error: <console>:35: error: type mismatch;
 found   : String(", ")
 required: org.apache.spark.sql.Column
           select(concat(col("first_name"), ", ", col("last_name"))).
                                            ^


In [35]:
// Referring columns using Data Frame
employeesDF.
    select(concat(employeesDF("first_name"), ", ", employeesDF("last_name"))).
    show

Unknown Error: <console>:35: error: type mismatch;
 found   : String(", ")
 required: org.apache.spark.sql.Column
           select(concat(employeesDF("first_name"), ", ", employeesDF("last_name"))).
                                                    ^


In [36]:
employeesDF.
    select(concat("first_name", ", ", "last_name")).
    show

Unknown Error: <console>:34: error: type mismatch;
 found   : String("first_name")
 required: org.apache.spark.sql.Column
           select(concat("first_name", ", ", "last_name")).
                         ^
<console>:34: error: type mismatch;
 found   : String(", ")
 required: org.apache.spark.sql.Column
           select(concat("first_name", ", ", "last_name")).
                                       ^
<console>:34: error: type mismatch;
 found   : String("last_name")
 required: org.apache.spark.sql.Column
           select(concat("first_name", ", ", "last_name")).
                                             ^


* Si nous passons les littéraux directement sous forme de chaîne ou de type numérique, cela échouera. Nous devons convertir les littéraux en type de colonne en utilisant la fonction `lit`.


In [37]:
// Using lit to use literals to derive new expressions
import org.apache.spark.sql.functions.{concat, col, lit}

In [38]:
employeesDF.
    select(concat(col("first_name"), lit(", "), col("last_name"))).
    show

+---------------------------------+
|concat(first_name, , , last_name)|
+---------------------------------+
|                     Scott, Tiger|
|                      Henry, Ford|
|                     Nick, Junior|
|                      Bill, Gomes|
+---------------------------------+



In [40]:
employeesDF.
    select(concat($"first_name", lit(", "), employeesDF("last_name")) alias ("Full Name")).
    show

+------------+
|   Full Name|
+------------+
|Scott, Tiger|
| Henry, Ford|
|Nick, Junior|
| Bill, Gomes|
+------------+



## Manipulation de chaînes - Conversion de casse et Longueur de chaîne
Vérifions les fonctions qui peuvent convertir la casse des valeurs de colonne qui sont de type chaîne et obtenir également la longueur.
* Convertir tous les caractères alphabétiques d'une chaîne en **majuscules** - `upper`
* Convertir tous les caractères alphabétiques d'une chaîne en **lowercase** - `lower`
* Convertir le premier caractère d'une chaîne en **uppercase** - `initcap`
* Obtenir le **nombre de caractères dans une chaîne** - `length`
* Toutes les 4 fonctions prennent un argument de type colonne.

### Application


* Utilisez les données des employés et créez un Data Frame.
* Appliquez les 4 fonctions sur **nationality** et voyez les résultats.

In [41]:
import org.apache.spark.sql.functions.{col, upper, lower, initcap, length}

In [42]:
employeesDF.
    select("employee_id", "nationality").
    withColumn("nationality_upper", upper(col("nationality"))).
    withColumn("nationality_lower", lower($"nationality")).
    withColumn("nationality_initcap", initcap(employeesDF("nationality"))).
    withColumn("nationality_length", length(col("nationality"))).
    show

+-----------+--------------+-----------------+-----------------+-------------------+------------------+
|employee_id|   nationality|nationality_upper|nationality_lower|nationality_initcap|nationality_length|
+-----------+--------------+-----------------+-----------------+-------------------+------------------+
|          1| united states|    UNITED STATES|    united states|      United States|                13|
|          2|         India|            INDIA|            india|              India|                 5|
|          3|united KINGDOM|   UNITED KINGDOM|   united kingdom|     United Kingdom|                14|
|          4|     AUSTRALIA|        AUSTRALIA|        australia|          Australia|                 9|
+-----------+--------------+-----------------+-----------------+-------------------+------------------+



## Manipulation de chaînes - substring

Nous pouvons extraire des sous-chaînes en utilisant la fonction  `substring`.
* Si nous traitons des **colonnes de longueur fixe**, nous utilisons `substring` pour extraire les informations.
* Voici quelques exemples de **colonnes de longueur fixe** et les cas d'utilisation pour lesquels nous extrayons généralement des informations.
 * Numéro de sécurité sociale à 9 chiffres. Nous extrayons généralement les 4 derniers chiffres et les fournissons aux applications de télévérification.
 * Numéro de carte de crédit à 16 chiffres. Nous utilisons généralement les 4 premiers chiffres pour identifier le fournisseur de carte de crédit et les 4 derniers chiffres aux fins de la télévérification.

* `substring` La fonction prend 3 arguments, **colonne**, **position**, **longueur**. Nous pouvons également fournir la position à partir de la fin en transmettant une valeur négative.


In [43]:
val s = "Hello World"

s = Hello World


Hello World

In [44]:
s.substring(0, 5)

Hello

In [45]:
s.substring(1, 4)

ell

In [46]:
s.substring(6, 11)

World

In [3]:
val l = List("X")

l = List(X)


List(X)

In [4]:
val df = l.toDF("dummy")

df = [dummy: string]


[dummy: string]

In [2]:
df.show

Unknown Error: <console>:24: error: not found: value df
       df.show
       ^


In [50]:
df.printSchema

root
 |-- dummy: string (nullable = true)



In [51]:
import org.apache.spark.sql.functions.{substring, lit}

In [54]:
df.select(substring(lit("Hello World"), 7, 5)).
    show

+--------+
|Resultat|
+--------+
|   World|
+--------+



In [56]:
df.select(substring(lit("Hello World"), -5, 5) alias ("Resultat")).
    show

+--------+
|Resultat|
+--------+
|   World|
+--------+



### Application

Effectuons quelques traitements pour extraire des informations à partir de chaînes de longueur fixe.
* Créez une liste pour les employés avec nom, ssn et numéro de téléphone.
* Format SSN **3 2 4** - Longueur fixe à 9 chiffres
* Format du numéro de téléphone - L'indicatif du pays est variable et le numéro de téléphone restant comporte 10 chiffres :
 * Code du pays - un à 3 chiffres
 * Indicatif régional - 3 chiffres
 * Préfixe du numéro de téléphone - 3 chiffres
 * Numéro de téléphone restant - 4 chiffres
 * Toutes les 4 parties sont séparées par des espaces
* Créez un Dataframe avec les noms de colonne name, ssn et phone_number
* Extraire les 4 derniers chiffres du numéro de téléphone.
* Extraire les 4 derniers chiffres du SSN.

In [58]:
val employees = List((1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                    )

employees = List((1,Scott,Tiger,1000.0,united states,+1 123 456 7890,123 45 6789), (2,Henry,Ford,1250.0,India,+91 234 567 8901,456 78 9123), (3,Nick,Junior,750.0,united KINGDOM,+44 111 111 1111,222 33 4444), (4,Bill,Gomes,1500.0,AUSTRALIA,+61 987 654 3210,789 12 6118))


List((1,Scott,Tiger,1000.0,united states,+1 123 456 7890,123 45 6789), (2,Henry,Ford,1250.0,India,+91 234 567 8901,456 78 9123), (3,Nick,Junior,750.0,united KINGDOM,+44 111 111 1111,222 33 4444), (4,Bill,Gomes,1500.0,AUSTRALIA,+61 987 654 3210,789 12 6118))

In [59]:
val employeesDF = employees.
    toDF("employee_id", "first_name",
         "last_name", "salary",
         "nationality", "phone_number",
         "ssn"
        )

employeesDF = [employee_id: int, first_name: string ... 5 more fields]


[employee_id: int, first_name: string ... 5 more fields]

In [60]:
employeesDF.show

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+--------------+----------------+-----------+



In [63]:
import org.apache.spark.sql.functions.substring
import spark.implicits._

In [67]:
employeesDF.
    select(substring($"phone_number", -4, 4) alias ("phone_last4"), 
           substring($"ssn", -4, 4) alias ("ssn_last4")
          ).
    show

+-----------+---------+
|phone_last4|ssn_last4|
+-----------+---------+
|       7890|     6789|
|       8901|     9123|
|       1111|     4444|
|       3210|     6118|
+-----------+---------+



In [64]:
employeesDF.
    select("employee_id", "phone_number", "ssn").
    withColumn("phone_last4", substring($"phone_number", -4, 4).cast("int")).
    withColumn("ssn_last4", substring($"ssn", 8, 4).cast("int")).
    show

+-----------+----------------+-----------+-----------+---------+
|employee_id|    phone_number|        ssn|phone_last4|ssn_last4|
+-----------+----------------+-----------+-----------+---------+
|          1| +1 123 456 7890|123 45 6789|       7890|     6789|
|          2|+91 234 567 8901|456 78 9123|       8901|     9123|
|          3|+44 111 111 1111|222 33 4444|       1111|     4444|
|          4|+61 987 654 3210|789 12 6118|       3210|     6118|
+-----------+----------------+-----------+-----------+---------+



In [65]:
employeesDF.
    select($"employee_id", $"phone_number", $"ssn", 
           substring($"phone_number", -4, 4).cast("int").alias("phone_last4"),
           substring($"ssn", 8, 4).cast("int").alias("ssn_last4")
          ).
    show

+-----------+----------------+-----------+-----------+---------+
|employee_id|    phone_number|        ssn|phone_last4|ssn_last4|
+-----------+----------------+-----------+-----------+---------+
|          1| +1 123 456 7890|123 45 6789|       7890|     6789|
|          2|+91 234 567 8901|456 78 9123|       8901|     9123|
|          3|+44 111 111 1111|222 33 4444|       1111|     4444|
|          4|+61 987 654 3210|789 12 6118|       3210|     6118|
+-----------+----------------+-----------+-----------+---------+



## Manipulation de chaînes - split
Voyons comment nous pouvons extraire des sous-chaînes en utilisant `split`.
* Si nous traitons des **colonnes de longueur variable** avec un **délimiteur**, nous utilisons `split` pour extraire les informations.
* oici quelques exemples de **colonnes de longueur variable** et les cas d'utilisation pour lesquels nous extrayons généralement des informations.
 * Adresse où nous stockons le numéro de maison, le nom de la rue, la ville, l'état et le code postal séparés par des virgules. Nous voudrons peut-être extraire la ville et l'état pour les rapports démographiques.
* `split` prend 2 arguments, **colonne** et **délimiteur**.
* `split` convertit chaque chaîne en tableau et nous pouvons accéder aux éléments en utilisant l'index.

In [9]:
val df = List("X").toDF("Dummy")

df = [Dummy: string]


[Dummy: string]

In [10]:
import org.apache.spark.sql.functions.{split, lit}

In [13]:
df.select(split(lit("Hello World, how are you"), " ") as "titre").
    show(false)

+------------------------------+
|titre                         |
+------------------------------+
|[Hello, World,, how, are, you]|
+------------------------------+



In [14]:
df.select(split(lit("Hello World, how are you"), " ")(2) alias "Split").
    show(false)

+-----+
|Split|
+-----+
|how  |
+-----+



### Application
Effectuons quelques traitements pour extraire des informations à partir de chaînes de longueur fixe ainsi que de chaînes de longueur variable délimitées.
* Créez une liste pour les employés avec nom, ssn et numéro de téléphone.
* Format du numéro de téléphone - L'indicatif du pays est variable et le numéro de téléphone restant comporte 10 chiffres :
 * Code du pays - un à 3 chiffres
 * Indicatif régional - 3 chiffres
 * Préfixe du numéro de téléphone - 3 chiffres
 * Numéro de téléphone restant - 4 chiffres
 * Toutes les 4 parties sont séparées par des espaces
* Créez un Dataframe avec les noms de colonne name, ssn et phone_number
* Extraire les 4 derniers chiffres du numéro de téléphone.
* Extraire les 4 derniers chiffres du SSN.

In [74]:
import org.apache.spark.sql.functions.split
import spark.implicits._

In [75]:
employeesDF.
    select("employee_id", "phone_number", "ssn").
    withColumn("area_code", split($"phone_number", " ")(1).cast("int")).
    withColumn("phone_last4", split($"phone_number", " ")(3).cast("int")).
    withColumn("ssn_last4", split($"ssn", " ")(2).cast("int")).
    show

+-----------+----------------+-----------+---------+-----------+---------+
|employee_id|    phone_number|        ssn|area_code|phone_last4|ssn_last4|
+-----------+----------------+-----------+---------+-----------+---------+
|          1| +1 123 456 7890|123 45 6789|      123|       7890|     6789|
|          2|+91 234 567 8901|456 78 9123|      234|       8901|     9123|
|          3|+44 111 111 1111|222 33 4444|      111|       1111|     4444|
|          4|+61 987 654 3210|789 12 6118|      987|       3210|     6118|
+-----------+----------------+-----------+---------+-----------+---------+



In [76]:
employeesDF.
    select($"employee_id", $"phone_number", $"ssn", 
           split($"phone_number", " ")(1).cast("int").alias("area_code"),
           split($"phone_number", " ")(3).cast("int").alias("phone_last"),
           split($"ssn", " ")(2).cast("int").alias("ssn_last4")
          ).
    show

+-----------+----------------+-----------+---------+----------+---------+
|employee_id|    phone_number|        ssn|area_code|phone_last|ssn_last4|
+-----------+----------------+-----------+---------+----------+---------+
|          1| +1 123 456 7890|123 45 6789|      123|      7890|     6789|
|          2|+91 234 567 8901|456 78 9123|      234|      8901|     9123|
|          3|+44 111 111 1111|222 33 4444|      111|      1111|     4444|
|          4|+61 987 654 3210|789 12 6118|      987|      3210|     6118|
+-----------+----------------+-----------+---------+----------+---------+



## Manipulation de chaînes - Concaténation de chaînes
Voyons comment concaténer des chaînes à l'aide de la fonction `concat`.
* Nous pouvons passer un nombre variable de chaînes à la fonction `concat`.
* Il renverra une chaîne concaténant toutes les chaînes.
* Si nous devons concaténer un littéral entre les chaînes, nous devons utiliser la fonction `lit`.

* Créez une nouvelle colonne nommée **full_name** en concaténant **first_name** et **last_name**.

In [77]:
import org.apache.spark.sql.functions.{concat, lit}
import spark.implicits._

In [78]:
employeesDF.
    withColumn("full_name", concat($"first_name", $"last_name")).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn| full_name|
+-----------+----------+---------+------+--------------+----------------+-----------+----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|ScottTiger|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123| HenryFord|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|NickJunior|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118| BillGomes|
+-----------+----------+---------+------+--------------+----------------+-----------+----------+



* Ajoutez une **virgule suivie d'un espace** entre **first_name** et **last_name**.

In [80]:
employeesDF.
    withColumn("full_name", concat($"first_name", lit(", "), $"last_name")).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+------------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|   full_name|
+-----------+----------+---------+------+--------------+----------------+-----------+------------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|Scott, Tiger|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123| Henry, Ford|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|Nick, Junior|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118| Bill, Gomes|
+-----------+----------+---------+------+--------------+----------------+-----------+------------+



## Manipulation de chaînes - Padding
Voyons comment remplir des caractères au début ou à la fin des chaînes.
* Nous complétons généralement les caractères pour créer des valeurs ou des enregistrements de longueur fixe.
* Les valeurs ou enregistrements de longueur fixe sont largement utilisés dans les systèmes basés sur les mainframes.
* La longueur de chaque champ dans les enregistrements de longueur fixe est prédéterminée et si la valeur du champ est inférieure à la longueur prédéterminée, nous remplissons avec un caractère standard.
* En termes de champs numériques, nous remplissons avec zéro sur le côté gauche ou avant. Pour les champs non numériques, nous remplissons avec un caractère standard à la fin ou à droite.
* Nous utilisons `lpad` pour remplir une chaîne avec un caractère spécifique en tête ou à gauche et `rpad` pour remplir en fin ou à droite.
* Lpad et rpad, prennent 3 arguments - colonne ou expression, longueur désirée et le caractère de remplissage.

In [83]:
val df = List("X").toDF("dummy")

df = [dummy: string]


[dummy: string]

In [84]:
import org.apache.spark.sql.functions.{lit, lpad}

In [85]:
df.select(lpad(lit("Hello"), 10, "-").alias("dummy")).show

+----------+
|     dummy|
+----------+
|-----Hello|
+----------+



In [86]:
df.select(lpad(lit(2), 2, "0").alias("dummy")).show

+-----+
|dummy|
+-----+
|   02|
+-----+



### Application

Effectuons simple application pour comprendre comment utiliser les fonctions de remplissage pour convertir nos données en enregistrements de longueur fixe.

* Prenons le dataframe **employeesDF**

* Utilisez les fonctions ** pad ** pour convertir chacun des champs en longueur fixe puis les concaténer. Voici les détails pour chacun des champs.
 * La longueur de employee_id doit être de 5 caractères et doit être complétée par des zéros.
 * La longueur du prénom et du nom doit être de 10 caractères et doit être complétée par - sur le côté droit.
 * La longueur du salaire doit être de 10 caractères et doit être complétée par des zéros.
 * La longueur de la nationalité doit être de 15 caractères et doit être complétée par - sur le côté droit.
 * La longueur du numéro de téléphone doit être de 17 caractères et doit être complétée par - sur le côté droit.
 * La longueur du ssn peut être laissée telle quelle. C'est 11 caractères.
 
* Créez un nouveau Dataframe **empFixedDF** avec le nom de colonne **employee**. Prévisualisez les données en désactivant la troncature.

In [90]:
import org.apache.spark.sql.functions.{lpad, rpad, concat}
import spark.implicits._

In [91]:
val empFixedDF = employeesDF.select(
    concat(
        lpad($"employee_id", 5, "0"),
        rpad($"first_name", 10, "-"),
        rpad($"last_name", 10, "-"),
        lpad($"salary", 10, "0"),
        rpad($"nationality", 15, "-"),
        rpad($"phone_number", 17, "-"),
        $"ssn"
    ).alias("employee")
)

empFixedDF = [employee: string]


[employee: string]

In [92]:
empFixedDF.show(false)

+------------------------------------------------------------------------------+
|employee                                                                      |
+------------------------------------------------------------------------------+
|00001Scott-----Tiger-----00001000.0united states--+1 123 456 7890--123 45 6789|
|00002Henry-----Ford------00001250.0India----------+91 234 567 8901-456 78 9123|
|00003Nick------Junior----00000750.0united KINGDOM-+44 111 111 1111-222 33 4444|
|00004Bill------Gomes-----00001500.0AUSTRALIA------+61 987 654 3210-789 12 6118|
+------------------------------------------------------------------------------+



## Manipulation de chaînes - c
Voyons comment supprimer les caractères indésirables de début et de fin  autour d'une chaîne.
* Nous utilisons généralement le Trimming pour supprimer les caractères inutiles des enregistrements de longueur fixe.
* Les enregistrements de longueur fixe sont largement utilisés.
* Dans le cadre d'un traitement, nous pouvons souhaiter supprimer les caractères de début ou de fin tels que 0 dans le cas de types numériques et d'espace ou certains caractères standard dans le cas de types alphanumériques.
* Les fonctions de trimming Spark prennent la colonne comme argument et suppriment par defaut les espaces de début ou de fin.
* Supprimer les espaces vers la gauche - `ltrim`
* Supprimer les espaces vers la droite - `rtrim`
* Supprimer les espaces des deux côtés - `trim`
* Nous pouvons également couper d'autres caractères que les espaces en utilisant ces fonctions de coupe.

In [93]:
import org.apache.spark.sql.functions.{col, ltrim, rtrim, trim}

In [94]:
val l = List("   Hello.    ")

l = List("   Hello.    ")


List("   Hello.    ")

In [95]:
val df = l.toDF("dummy")

df = [dummy: string]


[dummy: string]

In [96]:
df.withColumn("ltrim", ltrim(col("dummy"))).
    withColumn("rtrim", rtrim(rtrim(col("dummy")), ".")).
    withColumn("trim", trim(trim(col("dummy")), ".")).
    show()

+-------------+----------+--------+-----+
|        dummy|     ltrim|   rtrim| trim|
+-------------+----------+--------+-----+
|   Hello.    |Hello.    |   Hello|Hello|
+-------------+----------+--------+-----+



In [ ]:
df.withColumn("ltrim", ltrim(col("dummy"))).
    withColumn("rtrim", rtrim(rtrim(col("dummy")), ".")).
    withColumn("trim", trim(trim(col("dummy")), ".")).
    show()